*Please note, this directory is independent of the edward package such that requirements must be downloaded seperately and are not provided in `requirements.txt`.

# Plink v. Edward: PCA

## Time
Run `plink_edward.sh` with input file of choice

## Accuracy
1. Compare html output of Edward to code below to read in Plink output

In [1]:
from cyvcf2 import VCF
import numpy as np 

In [15]:
from cyvcf2 import VCF
import numpy as np 

vcf_path = '../example_data/friend_genome.vcf.gz'
vcf_path = '../example_data/gtdata_1000Genomes.vcf.gz'
vcf_path = '../example_data/gtdata_1000Genomes_pruned.vcf.gz'
gt_array = []
print(vcf_path)
for variant in VCF(vcf_path):
    gt = [sum(genotypes[:-1]) for genotypes in variant.genotypes]
    # variant.genotypes is list containing [0, 0, True], [1, 1, True], [0, 1, False], ... 
    # where the first 2 elements are the genotype and the third element is a boolean indicating
    # whether that genotype is phased or not (for now I'm ignoring this but it may be important
    # to consider later. 
    # 
    # NOTE: some of the genotypes only have one allele and some have negative alleles 
    # whatever that even means. For now I'm just gonna keep going but something to investigate.
    gt_array.append(gt)
gt_array = np.array(gt_array) # convert to numpy ndarray
row, col = gt_array.shape


../example_data/gtdata_1000Genomes_pruned.vcf.gz


[E::vcf_parse_format] Number of columns at 2:212106 does not match the number of samples (2149 vs 2504)


Exception: error parsing variant with `htslib::bcf_read` error-code: 0 and ret: -2

In [16]:
vcf = VCF(vcf_path)
for variant in vcf:
    print(variant)
    print(variant.genotypes)

2	90138	rs6759760	C	T	100	PASS	AC=340;AF=0.0678914;AN=5008;NS=2504;DP=14894;EAS_AF=0.0417;AMR_AF=0.0144;AFR_AF=0.2133;EUR_AF=0.001;SAS_AF=0.0051;AA=-|||;VT=SNP	GT	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|0	0|1	0|0	0|1	0|0	0|0	0|0	0|0	0|0	0|0	1|0	0|0	0|0	0

[E::vcf_parse_format] Number of columns at 2:212106 does not match the number of samples (2149 vs 2504)


Exception: error parsing variant with `htslib::bcf_read` error-code: 0 and ret: -2